In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict

In [2]:
class NetworkSimulator:
    def __init__(self):
        self.G = nx.Graph()
        self.packet_size = 800  # bits per packet as given
        self.setup_topology()
        self.setup_traffic_matrix()
        self.flows = defaultdict(float)
        self.delays = defaultdict(list)
        
    def setup_topology(self):
        # Create nodes A through G
        nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
        for node in nodes:
            self.G.add_node(node)
            
        # We'll start with a fully connected mesh - capacities can be adjusted
        for i in range(len(nodes)):
            for j in range(i+1, len(nodes)):
                self.G.add_edge(nodes[i], nodes[j])
    
    def setup_traffic_matrix(self):
        # Traffic matrix in Mbps as given
        self.traffic_matrix = {
            'A': {'B': 900, 'C': 600, 'D': 500, 'E': 700, 'F': 800, 'G': 200},
            'B': {'A': 900, 'C': 300, 'D': 200, 'E': 900, 'F': 500, 'G': 300},
            'C': {'A': 600, 'B': 300, 'D': 700, 'E': 100, 'F': 400, 'G': 500},
            'D': {'A': 500, 'B': 200, 'C': 700, 'E': 400, 'F': 700, 'G': 900},
            'E': {'A': 700, 'B': 900, 'C': 100, 'D': 400, 'F': 500, 'G': 400},
            'F': {'A': 800, 'B': 500, 'C': 400, 'D': 700, 'E': 500, 'G': 200},
            'G': {'A': 200, 'B': 300, 'C': 500, 'D': 900, 'E': 400, 'F': 200}
        }
        
    def calculate_packets_per_second(self, traffic_mbps):
        # Convert Mbps to bps and divide by packet size
        return (traffic_mbps * 1_000_000) / self.packet_size
    
    def calculate_link_utilization(self, capacity_mbps):
        utilization = {}
        for (src, dst), flow in self.flows.items():
            link = tuple(sorted([src, dst]))
            packets_per_sec = self.calculate_packets_per_second(flow)
            utilization[link] = (packets_per_sec * self.packet_size) / (capacity_mbps * 1_000_000)
        return utilization
    
    def calculate_link_delay(self, link, capacity_mbps):
        # M/M/1 queue delay calculation
        flow = self.flows[link]
        packets_per_sec = self.calculate_packets_per_second(flow)
        service_rate = (capacity_mbps * 1_000_000) / self.packet_size  # packets/sec
        
        if packets_per_sec >= service_rate:
            return float('inf')
            
        utilization = packets_per_sec / service_rate
        avg_delay = 1 / (service_rate - packets_per_sec)  # seconds
        return avg_delay * 1000  # convert to ms
    
    def simulate_ospf(self, capacity_mbps):
        self.flows.clear()
        paths = dict(nx.all_pairs_dijkstra_path(self.G))
        
        # Route traffic according to shortest paths
        for src in self.traffic_matrix:
            for dst, traffic in self.traffic_matrix[src].items():
                path = paths[src][dst]
                for i in range(len(path)-1):
                    link = tuple(sorted([path[i], path[i+1]]))
                    self.flows[link] += traffic
                    
        return self.calculate_metrics(capacity_mbps)
    
    def simulate_rip(self, capacity_mbps):
        self.flows.clear()
        paths = dict(nx.all_pairs_shortest_path(self.G))
        
        # Route traffic based on hop count
        for src in self.traffic_matrix:
            for dst, traffic in self.traffic_matrix[src].items():
                path = paths[src][dst]
                for i in range(len(path)-1):
                    link = tuple(sorted([path[i], path[i+1]]))
                    self.flows[link] += traffic
                    
        return self.calculate_metrics(capacity_mbps)
    
    def calculate_metrics(self, capacity_mbps):
        metrics = {
            'utilization': self.calculate_link_utilization(capacity_mbps),
            'delays': {},
            'avg_hops': self.calculate_average_hops(),
            'end_to_end_delay': self.calculate_average_end_to_end_delay(capacity_mbps)
        }
        
        for link in self.flows:
            metrics['delays'][link] = self.calculate_link_delay(link, capacity_mbps)
            
        return metrics
    
    def calculate_average_hops(self):
        paths = dict(nx.all_pairs_shortest_path(self.G))
        total_hops = 0
        total_pairs = 0
        
        for src in paths:
            for dst in paths[src]:
                if src != dst:
                    total_hops += len(paths[src][dst]) - 1
                    total_pairs += 1
                    
        return total_hops / total_pairs if total_pairs > 0 else 0
    
    def calculate_average_end_to_end_delay(self, capacity_mbps):
        paths = dict(nx.all_pairs_shortest_path(self.G))
        total_delay = 0
        total_pairs = 0
        
        for src in paths:
            for dst in paths[src]:
                if src != dst:
                    path = paths[src][dst]
                    path_delay = 0
                    
                    for i in range(len(path)-1):
                        link = tuple(sorted([path[i], path[i+1]]))
                        path_delay += self.calculate_link_delay(link, capacity_mbps)
                        
                    total_delay += path_delay
                    total_pairs += 1
                    
        return total_delay / total_pairs if total_pairs > 0 else 0

In [3]:
sim = NetworkSimulator()
capacities_to_test = [2000, 3000, 4000, 5000]

In [4]:
print("Testing different link capacities:")
for capacity in capacities_to_test:
    print(f"\nTesting capacity: {capacity} Mbps")
    
    ospf_metrics = sim.simulate_ospf(capacity)
    print("\nOSPF Metrics:")
    print(f"Average End-to-End Delay: {ospf_metrics['end_to_end_delay']:.3f} ms")
    print(f"Average Hops: {ospf_metrics['avg_hops']:.2f}")
    print("Average Link Utilization:", 
          sum(ospf_metrics['utilization'].values()) / len(ospf_metrics['utilization']))
    
    rip_metrics = sim.simulate_rip(capacity)
    print("\nRIP Metrics:")
    print(f"Average End-to-End Delay: {rip_metrics['end_to_end_delay']:.3f} ms")
    print(f"Average Hops: {rip_metrics['avg_hops']:.2f}")
    print("Average Link Utilization:", 
          sum(rip_metrics['utilization'].values()) / len(rip_metrics['utilization']))

Testing different link capacities:

Testing capacity: 2000 Mbps

OSPF Metrics:
Average End-to-End Delay: 0.001 ms
Average Hops: 1.00
Average Link Utilization: 0.5095238095238095

RIP Metrics:
Average End-to-End Delay: 0.001 ms
Average Hops: 1.00
Average Link Utilization: 0.5095238095238095

Testing capacity: 3000 Mbps

OSPF Metrics:
Average End-to-End Delay: 0.000 ms
Average Hops: 1.00
Average Link Utilization: 0.33968253968253964

RIP Metrics:
Average End-to-End Delay: 0.000 ms
Average Hops: 1.00
Average Link Utilization: 0.33968253968253964

Testing capacity: 4000 Mbps

OSPF Metrics:
Average End-to-End Delay: 0.000 ms
Average Hops: 1.00
Average Link Utilization: 0.25476190476190474

RIP Metrics:
Average End-to-End Delay: 0.000 ms
Average Hops: 1.00
Average Link Utilization: 0.25476190476190474

Testing capacity: 5000 Mbps

OSPF Metrics:
Average End-to-End Delay: 0.000 ms
Average Hops: 1.00
Average Link Utilization: 0.20380952380952386

RIP Metrics:
Average End-to-End Delay: 0.000 ms
